In [2]:
import sys
sys.path.append('../../')

from MPRA_predict.utils import *

from pyfaidx import Fasta
from Bio import pairwise2
from Bio.pairwise2 import format_alignment, align

/home/hxcai/anaconda3/envs/torch/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# 分析description

# 12:63513920:G:A:A:wC  G->A
# 12:63513920:G:A:R:wC  G->G

# 2:145626403:A:C:R:wC  A->A
# 2:145626403:A:C:A:wC  A->C

# 第三个是A的话,替换,alt
# 第三个是R的话,不变,ref

In [4]:
genome = Fasta('/home/hxcai/genome/hg19.fa')

seq1 = 'CATAAAGGGCTGAACATGCTGTTGAAAAAATGTAGATATAAAAGTTCATAAGAAATATTGATTCAATTTTTATGACTATGTAATTCTCATTTTTTTCAGATGCCATGTCAAAATATATTTGTTCCTCTTCTAGTTTTAGGATAATGAGACTGCTATGTGATATAACTACTTGGATGAATATTATCAGAAATCTAGATTTA'
seq2 = 'CATAAAGGGCTGAACATGCTGTTGAAAAAATGTAGATATAAAAGTTCATAAGAAATATTGATTCAATTTTTATGACTATGTAATTCTCATTTTTTTCAGGTGCCATGTCAAAATATATTTGTTCCTCTTCTAGTTTTAGGATAATGAGACTGCTATGTGATATAACTACTTGGATGAATATTATCAGAAATCTAGATTTA'
seq3 = genome['chr12'][63513820:63514020].seq
seq4 = seq_rc(seq3)

print(seq1)
print(seq2)
print(seq3)
print(seq4)

alignments = align.globalms(seq1, seq3, 2, 0, -.5, -.1)
print(format_alignment(*alignments[0]))
alignments = align.globalms(seq1, seq4, 2, 0, -.5, -.1)
print(format_alignment(*alignments[0]))
alignments = align.globalms(seq2, seq3, 2, 0, -.5, -.1)
print(format_alignment(*alignments[0]))
alignments = align.globalms(seq2, seq4, 2, 0, -.5, -.1)
print(format_alignment(*alignments[0]))

# 1和3只差一个替换
# 2和3完全一样
# Siraj和Gosai数据集的位置是hg19

CATAAAGGGCTGAACATGCTGTTGAAAAAATGTAGATATAAAAGTTCATAAGAAATATTGATTCAATTTTTATGACTATGTAATTCTCATTTTTTTCAGATGCCATGTCAAAATATATTTGTTCCTCTTCTAGTTTTAGGATAATGAGACTGCTATGTGATATAACTACTTGGATGAATATTATCAGAAATCTAGATTTA
CATAAAGGGCTGAACATGCTGTTGAAAAAATGTAGATATAAAAGTTCATAAGAAATATTGATTCAATTTTTATGACTATGTAATTCTCATTTTTTTCAGGTGCCATGTCAAAATATATTTGTTCCTCTTCTAGTTTTAGGATAATGAGACTGCTATGTGATATAACTACTTGGATGAATATTATCAGAAATCTAGATTTA
CATAAAGGGCTGAACATGCTGTTGAAAAAATGTAGATATAAAAGTTCATAAGAAATATTGATTCAATTTTTATGACTATGTAATTCTCATTTTTTTCAGGTGCCATGTCAAAATATATTTGTTCCTCTTCTAGTTTTAGGATAATGAGACTGCTATGTGATATAACTACTTGGATGAATATTATCAGAAATCTAGATTTA
TAAATCTAGATTTCTGATAATATTCATCCAAGTAGTTATATCACATAGCAGTCTCATTATCCTAAAACTAGAAGAGGAACAAATATATTTTGACATGGCACCTGAAAAAAATGAGAATTACATAGTCATAAAAATTGAATCAATATTTCTTATGAACTTTTATATCTACATTTTTTCAACAGCATGTTCAGCCCTTTATG
CATAAAGGGCTGAACATGCTGTTGAAAAAATGTAGATATAAAAGTTCATAAGAAATATTGATTCAATTTTTATGACTATGTAATTCTCATTTTTTTCAGATGCCATGTCAAAATATATTTGTTCCTCTTCTAGTTTTAGGATAATGAGACTGCTATGTGATATAACTACTTGGATGAATATTATCAGAAATCTAGA

In [5]:
df = pd.read_csv('../../data/SirajMPRA/SirajMPRA_562654.csv')
genome = Fasta('/home/hxcai/genome/hg19.fa')

# df = df[df['is_ref']].reset_index(drop=True)
flags = np.zeros(len(df), dtype=bool)

for i, row in tqdm(df.iterrows(), total=len(df)):
    chr, pos, ref, alt = row['chr'], row['pos'], row['ref'], row['alt']
    genome_seq = genome[chr][pos-100:pos+100].seq.upper()
    data_seq = row['seq'].upper()
    if genome_seq == data_seq:
        flags[i] = True

print(flags.sum())

# 每条len200的序列都正确对应hg19的序列

100%|██████████| 562654/562654 [00:13<00:00, 42794.36it/s]

281327


In [9]:
df = pd.read_csv('../../data/SirajMPRA/SirajMPRA_562654.csv')
df = df[df['is_ref']].reset_index(drop=True)
df[:100].to_csv('../../data/SirajMPRA/SirajMPRA_100.csv', index=False)

In [6]:
from pyliftover import LiftOver

lo = LiftOver('hg19', 'hg38')

pos_38 = np.zeros(len(df), dtype=int)
for i, row in tqdm(df.iterrows(), total=len(df)):
    chr, pos = row['chr'], row['pos']
    converted = lo.convert_coordinate(chr, pos)
    # 检查转换结果是否为空
    if converted and len(converted) > 0:
        pos_38[i] = int(converted[0][1]) # 成功转换的新位置
df['pos_38'] = pos_38

df

100%|██████████| 562654/562654 [00:09<00:00, 56955.33it/s]


,description,seq,A549,HepG2,K562,SKNSH,HCT116,is_ref,chr,pos,ref,alt,start,end,pos_38
0,chr1:14677:G:A,AGTGCTGGTTCCGTCACCCCCTCCCAAGGAAGTAGGTCTGAGCAGC...,NaN,0.667516,-0.704147,0.538641,0.159069,True,chr1,14677,G,A,14577,14777,14677
1,chr1:14677:G:A,AGTGCTGGTTCCGTCACCCCCTCCCAAGGAAGTAGGTCTGAGCAGC...,NaN,0.415984,0.554099,0.441904,-0.288019,False,chr1,14677,G,A,14577,14777,14677
2,chr1:63697:T:C,CATACATTTTTATTCTGGTCACTGTCCAACGACATTCCTCAAATGA...,NaN,1.269937,-0.391222,0.752052,1.100437,True,chr1,63697,T,C,63597,63797,63697
3,chr1:63697:T:C,CATACATTTTTATTCTGGTCACTGTCCAACGACATTCCTCAAATGA...,NaN,1.151995,-0.659023,0.204721,1.050619,False,chr1,63697,T,C,63597,63797,63697
4,chr1:64764:C:T,AATAGACTCTAGTACTCTGTATTATGCAAAATTTGTCTATGTTACA...,NaN,-0.211076,-0.007174,0.499376,0.257572,True,chr1,64764,C,T,64664,64864,64764
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
562649,chrX:155233098:T:C,ATCATTTACTGGTGACTGCCCTGCTAGGGTGTCAGCTGTCAGATCC...,NaN,0.615976,1.927090,0.798263,1.035440,False,chrX,155233098,T,C,155232998,155233198,156003433
562650,chrX:155233794:A:G,TGGTTACTGCAGGGGCAGGGTTTTGGCAGGAAATAAACATGCACGG...,NaN,1.409758,0.556338,2.843295,1.495515,True,chrX,155233794,A,G,155233694,155233894,156004129
562651,chrX:155233794:A:G,TGGTTACTGCAGGGGCAGGGTTTTGGCAGGAAATAAACATGCACGG...,NaN,0.894634,-0.072647,1.973661,0.794848,False,chrX,155233794,A,G,155233694,155233894,156004129
562652,chrX:155234730:T:C,TGGTGAGTTGCTTCTGTGCACACACTTTTGTTTATGAGTGTGCATG...,NaN,0.312172,0.165634,-0.041383,0.444472,True,chrX,155234730,T,C,155234630,155234830,156005065


In [9]:
genome_19 = Fasta('/home/hxcai/genome/hg19.fa')
genome_38 = Fasta('/home/hxcai/genome/hg38.fa')

flags = np.zeros(len(df), dtype=bool)
for i, row in tqdm(df.iterrows(), total=len(df)):
    chr, pos, pos_38 = row['chr'], row['pos'], row['pos_38']
    if row['is_ref']:
        genome_19_seq = genome_19[chr][pos-100:pos+100].seq.upper()
        genome_38_seq = genome_38[chr][pos_38-100:pos_38+100].seq.upper()
        if genome_19_seq == genome_38_seq:
            flags[i] = True

print(flags.sum())

100%|██████████| 281327/281327 [00:10<00:00, 27650.58it/s]

277505


In [6]:
(df['pos_38'] == 0).sum()

29

In [13]:
df.to_csv('../../data/SirajMPRA/SirajMPRA_281k.csv', index=False)